# LISTA 4 DE VISÃO COMPUTACIONAL
# LUCAS HERON SANTOS ANCHIETA
# RUAN TENÓRIO DE MELO

## 1. As fotografias em modo retrato se popularizaram nos  últimos anos. Elas consistem em segmentar foreground e background em uma fotografia, e borrar o background, simulando o efeito de uma câmera DLSR, como na imagem abaixo.
## O objetivo dessa questão é simular este efeito, usando mapas de disparidade gerados por duas imagens. Para um tutorial de como gerar mapas de disparidade usando OpenCV, veja https://docs.opencv.org/4.7.0/dd/d53/tutorial_py_depthmap.html. Após a aquisição do mapa de disparidade, você deve usá-lo adequadamente para detectar o foreground, aplicar um filtro gaussiano para borrar apenas o background, e combinar as duas partes para obter um efeito como na figura acima. Experimente seu algoritmo em três pares de imagens, em cenários diferentes, adquiridas através de pequenas variações de ponto de vista da câmera.

## 2. Leia o seguinte tutorial sobre reconstrução 3d, até a etapa de geração do arquivo PLY: https://medium.com/analytics-vidhya/depth-sensing-and-3d-reconstruction-512ed121aa60

## a) Experimente o código disponibilizado no tutorial em três exemplos distintos, gerando a nuvem de pontos e visualizando com o Meshlab.

## b) Descreva, com suas palavras, as etapas realizadas desde o fornecimento do par de imagens de entrada, até a geração da nuvem de pontos em formato PLY.

## 3. Usando a plataforma Kaggle, pesquise um base supervisionada de imagens que possa ser usada para treinar um classificador binário de imagens. Esta base deve conter exemplos de duas classes. Considere 70% das amostras para treino, e 30% para teste (método holdout).

## a) Usando OpenCV, extraia descritores HOG (Histogram of Gradients), e treine um classificador SVM. Exiba a acurácia atingida no conjunto de teste e a matriz de confusão. Exiba exemplos. Obs.: note que essa abordagem é semelhante ao detector Dalal-Triggs, porém aqui estamos usando-a para classificação de imagens. 

## b) Agora o objetivo é treinar uma CNN. Usando a abordagem de transferência de aprendizado (transfer learning), realize treinamentos usando modelos VGG16, ResNet50, e MobileNetV2 pré-treinados. Compare os resultados usando acurácias e matrizes de confusão. Exiba exemplos de acerto e erro.

## 4. O objetivo dessa questão é criar um contador de veículos em vídeos de uma rodovia. Você deve experimentar um modelo pré-treinado da YOLO, e um Single Shot MultiBox Detection (SSD).

## a) Estude a SSD, e descreva suas principais diferenças em relação à YOLO.

## b) Para cada um dos dois modelos, realize as detecções de veículos em cada frame do vídeo em anexo (classroom). Gere os vídeos com as bounding boxes detectadas.

## c) Para cada um dos dois modelos, plote um gráfico onde o eixo horizontal é o tempo (quadro do vídeo), e o vertical é a quantidade de carros detectada pelo modelo.